In [1]:
import math
import random
import string
import sys
import time
import numpy as np
import torch

import matplotlib.pyplot as plt
import src
from envs import build_env
#from src.evaluator import Evaluator
from src.slurm import init_signal_handler, init_distributed_mode
from t2.realtime_trainer import RealtimeTrainer
from src.utils import initialize_exp, str_diff
from t2.utils import get_parser, join_sai, to_sai_str

from t2.transformer import build_transformer
argv = [
        '--exp_name', 'first_train', 
        '--tasks', 'add_dataset', 
        '--n_enc_layers', '16', 
        '--n_heads', '4', 
        '--sinusoidal_embeddings', 'true', 
        '--num_workers', '4', 
        '--eval_onl', '0', 
        '--save_periodic', '0', 
        '--epoch_size', '10000', 
        '--batch_size', '32', 
        '--dropout', '0.1', 
        '--attention_dropout', '0.1', 
        '--emb_dim', '64', 
        '--bottleneck_dim', '64', 
        '--nn_output', '1', 
        '--input_seq_length', '47', 
        '--share_inout_emb', 'false'
]

parser = get_parser()

params = parser.parse_args(argv)
print(params)

bs = params.batch_size
#dim = params.emb_dim

init_distributed_mode(params)
logger = initialize_exp(params)

# CPU / CUDA
if params.cpu:
    assert not params.multi_gpu
else:
    assert torch.cuda.is_available()
    
src.utils.CUDA = not params.cpu

env = build_env(params)

# modules = build_transformer(env, params, 'cuda')
# trainer = RealtimeTrainer(modules, env, params)

trainers=[]
number_of_models=2
for i in range(number_of_models):
    params.my_device='cuda:' + str(i)
    trainer = RealtimeTrainer(build_transformer(env, params), env, params)
    trainers += [trainer]



INFO - 01/07/23 18:50:34 - 0:00:00 - ============ Initialized logger ============
INFO - 01/07/23 18:50:34 - 0:00:00 - accumulate_gradients: 1
                                     amp: -1
                                     attention_dropout: 0.1
                                     batch_size: 32
                                     beam_early_stopping: True
                                     beam_eval: False
                                     beam_length_penalty: 1
                                     beam_size: 1
                                     bottleneck_dim: 64
                                     clip_grad_norm: 5
                                     command: python F:\Programs\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py '-f' 'C:\Users\Dmytro\AppData\Roaming\jupyter\runtime\kernel-78e1afa4-519d-48c5-849b-7cf2f43b2a90.json' --exp_id "dmytro_job"
                                     cpu: False
                                     debug: False
               

SAIEnvironment created
Namespace(dump_path='./dumped/', exp_name='first_train', save_periodic=0, exp_id='dmytro_job', fp16=False, amp=-1, emb_dim=64, bottleneck_dim=64, nn_output=1, input_seq_length=47, n_enc_layers=16, n_dec_layers=4, n_heads=4, dropout=0.1, attention_dropout=0.1, share_inout_emb=False, sinusoidal_embeddings=True, env_base_seed=0, max_len=512, batch_size=32, optimizer='adam,lr=0.0001', clip_grad_norm=5, epoch_size=10000, max_epoch=100000, stopping_criterion='', validation_metrics='', accumulate_gradients=1, num_workers=4, same_nb_ops_per_batch=False, reload_size=-1, env_name='sai', tasks='add_dataset', beam_eval=False, beam_size=1, beam_length_penalty=1, beam_early_stopping=True, reload_model='', reload_checkpoint='', eval_only=False, eval_verbose=0, eval_verbose_print=False, debug_slurm=False, debug=False, cpu=False, local_rank=-1, master_port=-1)
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World si

INFO - 01/07/23 18:50:34 - 0:00:00 - Found 370 parameters in model.
INFO - 01/07/23 18:50:35 - 0:00:01 - Optimizers: model
INFO - 01/07/23 18:50:35 - 0:00:02 - Number of parameters (transformer): 1598630
INFO - 01/07/23 18:50:35 - 0:00:02 - Found 370 parameters in model.
INFO - 01/07/23 18:50:36 - 0:00:03 - Optimizers: model


In [2]:
#!nvidia-smi

In [3]:
def act(inp, trainer):
    inp = join_sai(inp)
    lst = []
    for _ in range(bs):
        x = inp
        lst += [x]
        
    return trainer.act(lst)[0]



In [4]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
a = join_sai(A)

In [5]:
start_time = time.time()

losses = []
iterations_number = 10

for it in range(iterations_number):
    for trainer in trainers:
            for _ in range(params.batch_size):
                loss = trainer.learn(a, a)

end_time = time.time()
elapsed_time = end_time - start_time
av_time = elapsed_time/iterations_number/len(trainers)

print(f'Av time per iteration per trainer: {elapsed_time/iterations_number:.2f} seconds')


F:\workspace\ai\Transformers1\src\optim.py:72: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\python_arg_parser.cpp:1418.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)
INFO - 01/07/23 18:50:38 - 0:00:04 - learning: av-score=0.0425531914893617, device=cuda:0
INFO - 01/07/23 18:50:38 - 0:00:05 - learning: av-score=0.033244680851063815, device=cuda:1
INFO - 01/07/23 18:50:38 - 0:00:05 - learning: av-score=0.043218085106382975, device=cuda:0
INFO - 01/07/23 18:50:39 - 0:00:05 - learning: av-score=0.048537234042553196, device=cuda:1
INFO - 01/07/23 18:50:39 - 0:00:05 - learning: av-score=0.10239361702127657, device=cuda:0
INFO - 01/07/23 18:50:39 - 0:00:05 - learning: av-score=0.22805851063829782, device=cuda:1
INFO - 01/07/23 18:50:39 - 0:00:06 - learning: av-score=0.39893617021276595, dev

Av time per iteration per trainer: 0.55 seconds


In [6]:
torch.cuda.get_device_name(next(trainers[0].modules['transformer'].te1.parameters()).device.index)

'NVIDIA GeForce RTX 3080 Ti'

In [7]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
B = 'KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE'
X = 'KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE'

#rnds = [gen_rnd_chars(xy_data_size_const) for _ in range(100)]

print(trainers[0].act(A))
print(trainers[0].act(B))

AssertionError: 

In [ ]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
B = 'KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE'
X = 'KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE'

losses = []
for it in range(2000):
    loss = trainer.learn(join_sai(A), join_sai(A))
    if loss[0]:
        l = loss[1].item()
        losses += [l]

plt.plot(losses)
plt.title('los s vs iteration')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.show()


In [ ]:
print(act(A))

In [ ]:
print(act(B))

In [ ]:
import torch.multiprocessing as mp

In [ ]:
mp.set_start_method('spawn', force=True)

In [ ]:
def m():
    while True:
        print(1)

In [ ]:
p = mp.Process(target=m, args=())

p.start()